In [1]:
library(ggplot2)
library(reshape2)
library(RColorBrewer)
suppressMessages(library(dplyr))
library(stringr)
suppressMessages(library(tidyr))
theme_set(theme_bw())
library(scales)
options(repr.plot.width=7, repr.plot.height=4)
isotypes = c('Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val')

paired_positions = c('X1.72'='1:72', 'X2.71'='2:71', 'X3.70'='3:70', 'X4.69'='4:69', 'X5.68'='5:68', 'X6.67'='6:67', 'X7.66'='7:66', 'X8.14'='*8:14', 'X9.23'='*9:23', 'X10.25'='10:25', 'X10.45'='*10:45', 'X11.24'='11:24', 'X12.23'='12:23', 'X13.22'='13:22', 'X15.48'='*15:48','X18.55'='*18:55', 'X19.56'='*19:56', 'X22.46'='*22:46', 'X26.44'='*26:44', 'X27.43'='27:43', 'X28.42'='28:42', 'X29.41'='29:41', 'X30.40'='30:40', 'X31.39'='31:39', 'X49.65'='49:65', 'X50.64'='50:64', 'X51.63'='51:63', 'X52.62'='52:62', 'X53.61'='53:61', 'X54.58'='*54:58')
paired_identities = c('GC', 'AU', 'UA', 'CG', 'GU', 'UG', 'PurinePyrimidine', 'PyrimidinePurine', 'StrongPair', 'WeakPair', 'AminoKeto', 'KetoAmino', 'Wobble', 'Paired', 'Bulge', 'Mismatched', 'Absent')

single_positions = c('X8'='8', 'X9'='9', 'X14'='14', 'X15'='15', 'X16'='16', 'X17'='17', 'X17a'='17a', 'X17b'='17b', 'X18'='18', 'X19'='19', 'X20'='20', 'X20a'='20a', 'X20b'='20b', 'X21'='21', 'X26'='26', 'X32'='32', 'X33'='33', 'X34'='34', 'X35'='35', 'X36'='36', 'X37'='37', 'X38'='38', 'X44'='44', 'X45'='45', 'X46'='46', 'X47'='47', 'X48'='48', 'X54'='54', 'X55'='55', 'X56'='56', 'X57'='57', 'X58'='58', 'X59'='59', 'X60'='60', 'X73'='73')
single_identities = c('A', 'C', 'G', 'U', 'Purine', 'Pyrimidine', 'Weak', 'Strong', 'Amino', 'Keto', 'B', 'D', 'H', 'V', 'Absent')

colors = c('A'='gray20', 'C'='gray20', 'G'='gray20', 'U'='gray20', 'Absent'='gray20', 'Purine'='gray20', 'Pyrimidine'='gray20', 'Weak'='gray20', 'Strong'='gray20', 'Amino'='gray20', 'Keto'='gray20', 'B'='gray50', 'D'='gray50', 'H'='gray50', 'V'='gray50', 'GC'='gray20', 'AU'='gray20', 'UA'='gray20', 'CG'='gray20', 'GU'='gray20', 'UG'='gray20', 'PurinePyrimidine'='gray20', 'PyrimidinePurine'='gray20', 'StrongPair'='gray20', 'WeakPair'='gray20', 'AminoKeto'='gray20', 'KetoAmino'='gray20', 'Wobble'='gray20', 'Paired'='gray20', 'Bulge'='gray20', 'Mismatched'='gray20')
fills = c('A'='#ffd92f', 'C'='#4daf4a', 'G'='#e41a1c', 'U'='#377eb8', 'Purine'='#ff8300', 'Pyrimidine'='#66c2a5', 'Weak'='#b3de69', 'Strong'='#fb72b2', 'Amino'='#c1764a', 'Keto'='#b26cbd', 'B'='#e5c494', 'D'='#ccebd5', 'H'='#ffa79d', 'V'='#a6cdea', 'Absent'='gray60', 'Mismatched'='gray30', 'Paired'='#ffffcc')

In [2]:
load('best-freqs.RData')
load('clade-isotype-specific.RData')
load('isotype-specific.RData')
load('consensus-IDEs.RData')
load('clade-isotype-specific-freqs.RData')

In [3]:
identities = read.delim('identities.tsv', sep='\t')
identities$quality = as.logical(identities$quality)
identities$restrict = as.logical(identities$restrict)
identities = identities %>% mutate(quality=quality & !restrict)

In [4]:
genome_table = read.delim('genome_table+.txt', sep='\t', stringsAsFactors=FALSE, header=FALSE, col.names=c("species_short", "species", "species_long", "domain", "clade"))

# Introduction

Strategy 1: find rejected (a) consensus, (b) isotype-specific, or (c) clade-isotype-specific elements. Set a cutoff at 95% presence, but fails (i) isotype check, (ii) clade check, or (iii) species check. Spin in previous work, if any. Bonus points if previous work _conflicts_ with consensus definition.

Strategy 2: highlight instances where previous work conflicts.

Some of these will have been covered in `first-pass-consensus`.

# Strategy 1A

## Find targets

In [5]:
code_groups = c('A'=1, 'C'=1, 'G'=1, 'U'=1, 'Absent'=1, 
                'Purine'=2, 'Pyrimidine'=2,
                'Weak'=3, 'Strong'=3, 'Amino'=3, 'Keto'=3,
                'B'=4, 'D'=4, 'H'=4, 'V'=4,
                'GC'=1, 'AU'=1, 'UA'=1, 'CG'=1, 'GU'=1, 'UG'=1,
                'StrongPair'=2, 'WeakPair'=2, 'Wobble'=2,
                'PurinePyrimidine'=3, 'PyrimidinePurine'=3, 'AminoKeto'=3, 'KetoAmino'=3,
                'Paired'=4, 'Mismatched'=4, 'Bulge'=4)

cutoff_freqs = data.frame()
for (cutoff in c(0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1.0)) {
  df = clade_iso_ac_freqs %>%
    group_by(positions, variable) %>%
    summarize(count=sum(value), freq=sum(value)/sum(total)) %>%
    filter(freq >= cutoff) %>%
    mutate(cutoff=as.character(cutoff)) %>%
    select(positions, variable, freq, cutoff) %>%
    group_by(positions) %>%
    arrange(code_groups[variable], desc(freq)) %>%
    filter(row_number(positions) == 1)
  if (nrow(cutoff_freqs) == 0) cutoff_freqs = df
  else cutoff_freqs = rbind(cutoff_freqs, df)
}

In [6]:
cutoff_freqs %>% 
  filter(positions %in% c(names(single_positions), names(paired_positions))) %>%
  select(positions, cutoff, variable) %>%
  spread(cutoff, variable)

,positions,0.5,0.6,0.7,0.8,0.9,0.95,0.99
1,X10.25,GC,GC,GC,Paired,Paired,Paired,Paired
2,X10.45,Mismatched,Mismatched,Mismatched,Mismatched,NA,NA,NA
3,X11.24,CG,CG,CG,PyrimidinePurine,PyrimidinePurine,PyrimidinePurine,Paired
4,X12.23,PyrimidinePurine,PyrimidinePurine,Paired,Paired,Paired,Paired,NA
5,X13.22,PyrimidinePurine,Paired,NA,NA,NA,NA,NA
6,X14,A,A,A,A,A,A,A
7,X15,G,G,G,Purine,Purine,Purine,Purine
8,X15.48,GC,GC,GC,PurinePyrimidine,PurinePyrimidine,Paired,NA
9,X16,U,U,U,Pyrimidine,Pyrimidine,B,NA
10,X17,Absent,Absent,Absent,NA,NA,NA,NA


In [7]:
consensus

,positions,identity
1,X10.25,Paired
2,X11.24,Paired
3,X14,A
4,X15,V
5,X18,V
6,X19,Purine
7,X21,H
8,X28.42,Paired
9,X29.41,Paired
10,X32,Pyrimidine


For strategy 1A, some good candidates here: 
- R9 (95%)
- Y11:R24 (99%)
- R15:Y48 (95%)
- Y16 (95%)
- G18, U55, and G18:U55 (99%)
- G19:C56 (99%)
- A21 and A14/A21 (99%)
- U33 (99%)
- R37 (99%)
- R46 (99%)
- G53:C61 (99%)
- U54:A58 (99%)
- U55 (99%)
- R57 (99%)
- Y60 (95%)

## R9 and R46

### What's known

- Marck and Grosjean have a purine for iMet and no consensus for elongator, though they are mostly R or V. R9 is also conserved in initiators for archaea/bacteria.
- 9-12-23 is a tertiary interaction. [Gautheret et al. (1995)](http://dx.doi.org/10.1006/jmbi.1995.0200) has these frequencies from the Sprinzl 1991 database: ![9-12-23 frequency matrix](figures/9-12-23-gautheret.png)

- Position 9 is known to be modified with m$^1$G (along with 37) in a wide range of eukaryotes. 

### Our data

- Here's our frequencies:

In [8]:
table(paste0(identities[identities$quality, ]$X12, ':', identities[identities$quality, ]$X23), identities[identities$quality, ]$X9)

     
         -    A    C    G    U
  A:A    0    5    1    7    0
  A:C    0   25    2  225    0
  A:G    0    2    1   14    0
  A:U    0  461  146   60    4
  C:A    0    7    0   56    1
  C:C    0    1    0    4    0
  C:G    0  223   57 6511   50
  C:U    0    0    0    6    0
  G:A    0    2    0    2    0
  G:C    1 1575    7 3980    5
  G:G    0    0    0    8    0
  G:U    0   13    1   31    0
  U:A    0 6345    6 1711   33
  U:C    0    1    0    2    0
  U:G    0   23    0   68    0
  U:U    0    5    0    1    0

#### Which isotypes/clades/species fail the consensus checks?

In [9]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X9') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable %in% c("G", "A", "Purine"))*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X9) %>% group_by(clade, species, isotype) %>%
  summarize(status=sum(X9 %in% c("G", "A", "Purine")/n()) >= 0.1,
            freq=sum(X9 %in% c("G", "A", "Purine")), ntRNAs=n()) %>%
  filter(!status)

,clade,isotype,status
1,Insecta,Glu,FALSE
2,Insecta,His,FALSE
3,Mammalia,His,FALSE
4,Spermatophyta,Glu,FALSE
5,Streptophyta,Glu,FALSE
6,Vertebrata,His,FALSE
7,Vertebrata,Phe,FALSE


,clade,species,isotype,status,freq,ntRNAs
1,Fungi,botrCine_B05_10,Tyr,FALSE,0,5
2,Fungi,crypGatt_WM276,Gly,FALSE,0,2
3,Fungi,crypNeof_VAR_GRUBII_H99,Gly,FALSE,0,2
4,Fungi,crypNeof_VAR_NEOFORMANS_B_3501,Gly,FALSE,0,2
5,Fungi,crypNeof_VAR_NEOFORMANS_JEC21,Gly,FALSE,0,2
6,Insecta,dm6,His,FALSE,0,5
7,Insecta,dp4,His,FALSE,0,5
8,Insecta,droAna3,His,FALSE,0,5
9,Insecta,droEre2,His,FALSE,0,6
10,Insecta,droGri2,His,FALSE,0,5


### What's new

The 9:12:23 ratios are a bit different. But it shows the same thing: there is some selectivity for what base triples are allowed, but enough tolerance that there are a few interactions that persist.

The default hypothesis is that the 9-23 interaction has a conserved interaction type that explains the frequencies shown. We can marginalize as such because 12:23 is always a WC pair. This is not the case - although some (R:R) have a a strong trans WC-Hoogsteen pair of hydrogen bonds, others (C:R) do not. The hoogsteen side of C is just two stable carbons unlikely to hydrogen bond.

Examining the frequencies _without_ histidine shows that the C9:U23 is conserved in His, and A9:U23 is conserved in Asp, for some clades. (See tertiary interactions figure).

In [10]:
table(identities[identities$quality, ]$X9, identities[identities$quality, ]$X23)

   
       -    A    C    G    U
  -    0    0    1    0    0
  A    0 6359 1602  248  479
  C    0    7    9   58  147
  G    0 1776 4211 6601   98
  U    0   34    5   50    4

So here's what we know.
- 12:23 is almost always WC paired, so we can marginalize it.
- 9 is typically a purine, though there are isotype- and clade-specific exceptions. For example, His and Asp deviate in 4 clades.
    - In the His case, C and U can interact via the 2-carbonyl [amino pairing](http://www.columbuslabs.org/wp-content/uploads/2008/03/basepairs.pdf).
- 9:23 is thought to be a _trans_ interaction with two hydrogen bonds.

**Conclusion 1**: Position 9 was thought to be a purine. Instead, it varies by isotype and clade. Much of the variation from R9 can be explained by looking at 9:23 as an clade/isotype discriminating tertiary interaction.

We can leave it at that. But we should also look into compensatory interactions if 9:23 is disrupted.

## Covariation for core D stem 3d pairs

#### tRNA covariation frequencies for 9:23, 22:46, and 10:45

In [11]:
df = table(paste0(identities[identities$quality, ]$X9, ':', identities[identities$quality, ]$X23),
           paste0(identities[identities$quality, ]$X46, ':', identities[identities$quality, ]$X22, ' / ', identities[identities$quality, ]$X45, ':', identities[identities$quality, ]$X10),
           identities[identities$quality, ]$isotype)

In [12]:
as.data.frame(df) %>% group_by(Var3) %>% filter(Freq > 50)

,Var1,Var2,Var3,Freq
1,A:A,G:G / G:G,Ala,1358
2,G:C,G:G / G:G,Ala,88
3,G:A,G:U / G:G,Ala,51
4,G:C,A:A / G:G,Arg,281
5,G:G,A:A / G:G,Arg,941
6,G:C,G:G / G:G,Arg,440
7,G:G,A:A / G:G,Asn,68
8,A:A,G:G / G:G,Asn,107
9,G:C,G:G / G:G,Asn,452
10,A:U,A:G / G:G,Asp,312


This is messy. For each isotype there may be a different tertiary interaction compensating for the lack of A:A. It seems that purine:purine interactions are unusually enriched among all three of these except in valine. 

![classical interaction structure](figures/3d-interactions-oliva.png)

#### Average number of purine:purine interactions by isotype and clade

In [13]:
RRs = c("A:A", "A:G", "G:G", "G:A")
identities %>% select(isotype, clade, quality, X9.23, X22.46, X10.45, X26.44) %>% 
  filter(quality) %>%
  rowwise() %>% 
  mutate(nRR=(X9.23 %in% RRs) + (X22.46 %in% RRs) + (X10.45 %in% RRs) + (X26.44 %in% RRs)) %>%
  group_by(isotype, clade) %>% 
  summarize(nRR=signif(mean(nRR), 3)) %>%
  spread(isotype, nRR)

Warning message:
: Grouping rowwise data frame strips rowwise nature

,clade,Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,Ile,iMet,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val
1,Fungi,2.85,3.26,3.52,3.05,3.16,1.45,1.85,1.92,2.82,3.46,2.39,1.97,3.52,3.85,3.85,2.03,1.78,3.84,3.02,3.63,2.31
2,Insecta,4,3.62,3,2,4,2,3.3,2.5,2.07,3.57,3,2,4,3,3.94,2.05,1.99,3.97,2.96,4,1.99
3,Mammalia,3.95,3.53,2.94,2.11,3.97,1.97,2.96,2.64,2.04,3.49,3,2,3.92,2.99,3.92,2,2.05,3.99,2.98,3.99,2.17
4,Nematoda,3.98,3.58,2.94,2,4,1.93,2.74,2.49,1.94,3.5,3,1.96,2.66,2.93,3.85,1.98,1.99,3.97,2.79,3.95,2
5,Spermatophyta,4,3.51,3.79,2.05,3.84,2.58,3.55,3.04,3,4,3,2,3.03,3.11,4,2,1.12,3.68,3,4,2
6,Streptophyta,3.98,3.45,3.73,1.97,3.6,2.35,3.39,3.25,3,3.99,3,2,3,2.97,3.97,2.06,1.27,3.73,3,3.78,2
7,Vertebrata,3.96,3.45,2.93,2,3.92,1.98,2.96,2.34,1.99,3.63,2.99,1.97,3.94,2.96,3.72,1.99,2.06,3.95,2.99,3.93,1.97


#### Average number of purine:purine interactions by position/isotype

In [14]:
RRs = c("A:A", "A:G", "G:G", "G:A")
identities %>% select(isotype, quality, X9.23, X22.46, X10.45, X26.44) %>% 
  filter(quality) %>%
  rowwise() %>% 
  mutate(X9.23=X9.23 %in% RRs, X22.46=X22.46 %in% RRs, X10.45=X10.45 %in% RRs, X26.44=X26.44 %in% RRs) %>%
  gather(position, RR, X9.23, X22.46, X10.45, X26.44, -isotype, -quality) %>%
  group_by(isotype, position) %>%
  summarize(RR=round(mean(RR), digits=1)) %>%
  spread(isotype, RR)

,position,Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,Ile,iMet,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val
1,X10.45,1,1,1,1,1,1,0.9,0.9,1,1,0.8,0,0.9,1,0.9,1,0.1,1,1,1,0.9
2,X22.46,0.9,1,1,1,1,0.8,0.9,0.9,0.9,1,1,0.9,1,1,1,0,0.9,1,1,1,0.1
3,X26.44,1,1,0.9,0.1,0.9,0,0.2,0.1,0.2,0.9,1,0,0.9,1,1,0,0,1,0.9,1,1
4,X9.23,0.9,0.6,0.3,0.2,1,0.1,0.8,0.6,0.2,0.7,0,1,1,0.1,1,1,0.9,1,0.1,1,0.1


#### Average number of interactions containing a purine by position/isotype

In [15]:
RRs = c("A:A", "A:G", "G:G", "G:A", "G:C", "C:G", "A:C", "C:A", "U:G", "G:U", "A:U", "U:A")
identities %>% select(isotype, quality, X9.23, X22.46, X10.45, X26.44) %>% 
  filter(quality) %>%
  rowwise() %>% 
  mutate(X9.23=X9.23 %in% RRs, X22.46=X22.46 %in% RRs, X10.45=X10.45 %in% RRs, X26.44=X26.44 %in% RRs) %>%
  gather(position, RR, X9.23, X22.46, X10.45, X26.44, -isotype, -quality) %>%
  group_by(isotype, position) %>%
  summarize(RR=round(mean(RR), digits=1)) %>%
  spread(isotype, RR)

,position,Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,Ile,iMet,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val
1,X10.45,1,1,1,1,1,1,1,0.9,1,1,1,0,1,1,1,1,0.1,1,1,1,1
2,X22.46,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,X26.44,1,1,1,0.9,1,0.1,0.4,1,0.4,1,1,1,1,1,1,1,1,1,1,1,1
4,X9.23,1,1,1,1,1,1,1,1,0.5,1,1,1,1,1,1,1,1,1,1,1,1


#### Are non-purine-containing interactions enriched for some other type of interaction?

In [16]:
identities %>% select(isotype, quality, X9.23, X22.46, X10.45, X26.44) %>% 
  filter(quality) %>%
  gather(position, identity, X9.23, X22.46, X10.45, X26.44, -isotype, -quality) %>%
  filter(!(identity %in% c(RRs, "G:-", "A:-"))) %>%
  group_by(position, isotype, identity) %>%
  summarize(count=n()) %>%
  filter(count > 20) %>%
  spread(identity, count, fill=0)

Warning message:
: attributes are not identical across measure variables; they will be dropped

,position,isotype,C:-,C:U,U:-,U:C,U:U
1,X10.45,Leu,45,0,0,0,0
2,X10.45,Ser,91,0,153,0,0
3,X26.44,Asp,0,0,0,0,43
4,X26.44,Gln,0,31,0,667,123
5,X26.44,Glu,0,0,0,550,102
6,X26.44,Gly,0,0,0,0,26
7,X26.44,His,0,0,0,172,0
8,X26.44,Ile,0,0,0,32,0
9,X9.23,His,0,147,0,0,0


They are not - but they are isotype-specific.

**Conclusion**: Tertiary interactions within this "tRNA core region" along the D stem are enriched in purines and purine-purine interactions, though the extent varies by isotype and position. Exceptions are isotype-specific.

## G18, U55, and G18:U55

#### Number of tRNAs without G18:U55

In [17]:
identities %>% select(clade, species, quality, X18, X55, X18.55) %>%
  filter(quality & X18.55 != "G:U") %>% nrow()

[1] 301

#### Which isotypes/clades/species fail the consensus checks?

In [18]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X18.55') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable == "GU")*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, quality, X18.55) %>%
  filter(quality) %>%
  group_by(clade, species, isotype) %>%
  summarize(status=sum((X18.55 == "G:U")/n()) >= 0.1,
            freq=sum(X18.55 == "G:U"), tRNAs=n()) %>%
  filter(!status)

,clade,isotype,status
1,Nematoda,Asn,FALSE
2,Vertebrata,Gly,FALSE


,clade,species,isotype,status,freq,tRNAs
1,Fungi,flamVelu_KACC42780,Pro,FALSE,0,1
2,Fungi,valsMali_03_8,Pro,FALSE,0,2


In [28]:
identities %>% select(clade, quality, isotype, X18.55) %>%
  filter(quality & ((clade == "Nematoda" & isotype == "Asn") | 
                    (clade == "Vertebrata" & isotype == "Gly"))) %>%
  group_by(isotype, clade) %>%
  mutate(total=n()) %>%
  group_by(isotype, clade, total, X18.55) %>%
  summarize(count=n()) %>%
  mutate(freq=count/total) %>%
  select(-total, -count) %>%
  spread(X18.55, freq, fill=0)

Adding missing grouping variables: `total`


,total,isotype,clade,A:U,G:C,G:G,G:U,-:U
1,17,Asn,Nematoda,0.1176471,0,0,0.8823529,0
2,250,Gly,Vertebrata,0,0.136,0.004,0.856,0.004


The isotype/clade check fails for nematode Asn at 88% and vertebrate Gly at 86%. 

Earlier, we had a vertebrate check fail because of the methods used to build the quality set. I've relaxed the species check constraints to 10% to better identify IDEs even when there are massively amplified tRNAs. Curiously, in danRer10, gasAcu1, and oryLat2, the majority of the non-G:U tRNAs contain a G:C. It's pretty likely that, given the high copy number, many of these tRNAs are actually pseudogenes. In addition, there are a decent amount of G18:U55 tRNAs to actually drive translation.

The two fungi listed here are rare exceptions to the rule. _Flammulina velutipes_ is the enokitake mushroom, and none of its proline tRNAs contain G18. _Valsa mali_ is a fungus that infects apple trees. It contains 4 proline tRNAs. The first two (Pro-AGG-1, Pro-CGG-1) score at 22 and 47, contain G18:U55, and are filtered out. The other two (Pro-TGG-1) score at 58, and contain C18:U55. So we have two main possibilities here.

1. G18:U55 is required for proper folding and recognition. Pro-CGG-1 is used as the main proline tRNA.
2. G18:U55 is not required. The synthetase is able to recognize some combination of Pro-CGG and Pro-TGG.

The evidence below points to option 2. TGG is a much more common anticodon than CGG. Pro-TGG also scores better, and contains a position 47. There are only two other fungal proline tRNAs without a position 47, both of which exist in _Encephalitozoon_ species, and both of which are supplemented by other high-scoring proline tRNAs.

#### Fungal proline anticodon usage /  Fungal proline position 47 incidence

In [19]:
identities %>% select(clade, quality, isotype, anticodon) %>%
  filter(quality & isotype == "Pro" & clade == "Fungi") %>%
  group_by(clade, isotype, anticodon) %>%
  summarize(count=n())

identities %>% select(clade, quality, isotype, X47) %>%
  filter(quality & isotype == "Pro" & clade == "Fungi") %>%
  group_by(clade, isotype, X47) %>%
  summarize(count=n()) %>%
  spread(X47, count)

identities %>% select(clade, quality, species, species_long, seqname, isotype, score, X47) %>%
  filter(quality & isotype == "Pro" & clade == "Fungi" & X47 == "-")

,clade,isotype,anticodon,count
1,Fungi,Pro,AGG,60
2,Fungi,Pro,CGG,29
3,Fungi,Pro,TGG,90


,clade,isotype,-,C,U
1,Fungi,Pro,2,53,124


,clade,quality,species,species_long,seqname,isotype,score,X47
1,Fungi,TRUE,enceHell_ATCC50504,Encephalitozoon hellem ATCC 50504,enceHell_ATCC50504_chrX.trna2,Pro,60.7,-
2,Fungi,TRUE,enceInte_ATCC50506,Encephalitozoon intestinalis ATCC 50506,enceInte_ATCC50506_chrX.trna2,Pro,63.4,-


## Y11:R24

#### Number of tRNAs without C11:G24 or U11:A24

In [20]:
identities %>% select(clade, species, quality, X11.24) %>%
  filter(quality & !(X11.24 %in% c("C:G", "U:A"))) %>% nrow()

[1] 393

#### Which isotypes/clades/species fail the consensus checks?

In [22]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X11.24') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable %in% c("CG", "UA", "PyrimidinePurine"))*count)/sum(count) == 1,
            freq=((variable %in% c("CG", "UA", "PyrimidinePurine"))*count)/sum(count)) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, quality, X11.24) %>%
  filter(quality) %>%
  group_by(clade, species, isotype) %>%
  summarize(status=sum((X11.24 %in% c("C:G", "U:A"))/n()) >= 0.1,
            freq=sum(X11.24 %in% c("C:G", "U:A")), tRNAs=n()) %>% 
  filter(!status)

,clade,isotype,status,freq
1,Fungi,Val,FALSE,0


,clade,species,isotype,status,freq,tRNAs


In [33]:
identities %>% select(clade, quality, isotype, X11.24) %>%
  filter(quality & clade == "Fungi" & isotype == "Val") %>%
  group_by(clade, isotype, X11.24) %>%
  summarize(count=n())

,clade,isotype,X11.24,count
1,Fungi,Val,A:U,1
2,Fungi,Val,C:G,31
3,Fungi,Val,G:C,1
4,Fungi,Val,U:A,184
5,Fungi,Val,U:G,87


Although pyrimidine:purine passes the species check, it doesn't meet the 90% threshold required in fungal valine tRNAs due to a high frequency of U11:G24 (though technically, it is also a pyrimidine:purine interaction).

### What's known

- [Ohana et al. 1994](http://dx.doi.org/10.1006/abbi.1994.1503) studied yeast Ser, which contains C11:G24. A switch to G11:C24 decreased aminoacylation efficiency to 78%.

- C11:G24, along with G10:C25, are vital for m$^2_2$G26, according to [Edqvist et al. 1992](https://doi.org/10.1093/nar/20.24.6575). However, other eukaryotes such as _Drosophila_ contain the modification even without these identities.

- U11:A24 prevents RNase P processing in _Drosophila_ ([Levinger et al. 1995](10.1074/jbc.270.32.18903)). It's noted that the tRNA probably folds correctly, and the recognition is stalled by "local helix deformation". 

- This identity element is also important in mitochondrial tRNAs ([Takeuchi et al. 2001)](http://www.jbc.org/content/276/23/20064.full)).

### What's new

#### Covariance of 10:25 and 11:24 with respect to U:G

In [64]:
identities %>% select(clade, isotype, quality, X10.25, X11.24) %>%
  filter(isotype == "Val") %>%
  mutate(clade=ifelse(clade == "Fungi", "Fungi", "Other")) %>%
  filter(X11.24 %in% c("U:A", "C:G", "U:G")) %>%
  group_by(clade) %>%
  mutate(total=n()) %>%
  group_by(clade, total, X10.25, X11.24) %>%
  summarize(count=n()) %>%
  mutate(freq=count/total) %>%
  filter(freq > 0.01) %>%
  select(-total, -count) %>%
  spread(X10.25, freq, fill=0)

Adding missing grouping variables: `total`


,total,clade,X11.24,G:C,G:U
1,846,Fungi,C:G,0.04373522,0
2,846,Fungi,U:A,0.4160757,0.04137116
3,846,Fungi,U:G,0.4787234,0
4,3862,Other,U:A,0.9093734,0.07353703


A whopping **48%** of fungal valine tRNAs contain G10:C25 and U11:G24. This is very different from other eukaryotic valines, which hovers around 91% for G10:C25 and U11:A24. This seems to be tolerated specifically in fungal valine tRNAs. This could potentially be a switch for modification or for cleavage.

## R15:Y48

#### Number of tRNAs without R15:Y48

In [67]:
identities %>% select(clade, species, quality, X18, X55, X18.55) %>%
  filter(quality & X18.55 != "G:U") %>% nrow()

[1] 301

#### Which isotypes/clades/species fail the consensus checks?

In [69]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X15.48') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable %in% c('GC', 'AU', 'PurinePyrimidine'))*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, quality, X15.48) %>%
  filter(quality) %>%
  group_by(clade, species, isotype) %>%
  summarize(status=sum((X15.48 %in% c('G:C', 'A:U'))/n()) >= 0.1,
            freq=sum(X15.48 %in% c('G:C', 'A:U')), tRNAs=n()) %>%
  filter(!status)

,clade,isotype,status
1,Mammalia,Lys,FALSE
2,Spermatophyta,Ala,FALSE
3,Streptophyta,Ala,FALSE
4,Streptophyta,Asp,FALSE
5,Streptophyta,Lys,FALSE
6,Vertebrata,iMet,FALSE


,clade,species,isotype,status,freq,tRNAs
1,Fungi,ashbGoss_ATCC10895,Glu,FALSE,0,2
2,Fungi,aspeFumi_AF293,Asp,FALSE,0,2
3,Fungi,aspeNidu_FGSC_A4,Asp,FALSE,0,1
4,Fungi,botrCine_B05_10,Phe,FALSE,0,4
5,Fungi,candGlab_CBS_138,Glu,FALSE,0,2
6,Fungi,candOrth_CO_90_125,Phe,FALSE,0,1
7,Fungi,crypGatt_WM276,Ala,FALSE,0,1
8,Fungi,crypNeof_VAR_GRUBII_H99,Ala,FALSE,0,1
9,Fungi,crypNeof_VAR_GRUBII_H99,Gln,FALSE,0,1
10,Fungi,crypNeof_VAR_NEOFORMANS_B_3501,Ala,FALSE,0,1


This one's the Levitt base pair, and has been shown in E. coli to tolerate different combinations other than R15:Y48. Looks like a typical clade/isotype-specific IDE.